# Problème

Comment mieux fusionner deux estimations ?

- Prédiction fournie par l’odométrie
- Estimation fournie par les perceptions

**On cherche à estimer l'état du robot à partir de son évolution et de mesures reliées à cet état.**

<table><tr>
<td> <img src='img/kalman_problem.png' /> </td>
<td> <img src='img/kalman_exemple.png' /> </td>
</table></tr>


# Filtrage de Kalman

- **Etat** : Position $X_t=(x,y,q)$  
- **Incertitude** : matrice de covariance $P$ (gaussienne)

<img src="img/kalman_filter.png" width=600>


# Modèle Linéaire


- Etat $X$ (dim $n$), contrôle $u$ (dim $l$), observation $Y$ (dim $k$)
- Matrices de covariance : $Q$ bruit sur $u$ et $P_Y$ bruit sur $Y$
- Evolution de l’état :   $ X_{t+1} = A X_t + B u_t $
- Observation de l’état : $ Y_t = H X_t $

**Filtre :**

**1)** Prédiction de l'état

$$\text{moyenne :  } X_t^* = A \hat{X}_{t-1} + B u_t $$

$$\text{variance :  } P_t^* = A \hat{P}_t A^T + B Q B^T $$

**2)** Prédiction de l'observation

$$ Y_t^* = H X_t^* $$

**3)** Correction de la prédiction

$$ \hat{X}_t = X_t^* + K(Y - Y_t^*) $$

$$ \hat{P}_t = P_t^* - K H P_t^* $$

$$ \text{avec le gain de Kalman : } K = P_t^* H^T (H P_t^* H^T + P_Y)^{-1} $$

In [ ]:
def Kalman_update(x_est, P_est, Y_mesure, A, B, H):
    
    # prediction de l'état
    x_pred = A @ x_est + B @ u
    P_pred = A @ P_est @ A.T + B @ Q_est @ B.T
    
    # prediction de l'observation
    Y_predit = H @ x_pred 
    
    # correction de la prediction
    K     = P_pred @ H.T @ inv(H @ P_pred @ H.T + P_Y_est)
    x_est = x_pred + K @ (Y_mesure - Y_predit)
    P_est = P_pred - K @ H @ P_pred
    
    return x_est, P_est

# Modèle non Linéaire

La prédiction et l'observation sont des fonctions non linéaires $f$ et $h$.   
$A$, $B$ et $H$ sont remplacées par les jacobiennes de $f$ et $h$

**1)** Prédiction de l'état

$$ X_t^* = f(\hat{X}_{t-1}, u_t) $$

$$ P_t^* = A \hat{P}_t A^T + B Q B^T $$

**2)** Prédiction de l'observation

$$ Y_t^* = h(X_t^*) $$

**3)** Correction de la prédiction

$$ \hat{X}_t = X_t^* + K(Y - Y_t^*) $$

$$ \hat{P}_t = P_t^* - K H P_t^* $$

$$ \text{avec le gain de Kalman : } K = P_t^* H^T (H P_t^* H^T + P_Y)^{-1} $$


In [1]:
def Kalman_NL_update(x_est, P_est, Y_mesure, f, h):
    
    # calcul les jacobiennes
    A = Jacobienne_fx(x,u)
    B = Jacobienne_fu(x,u)
    H = Jacobienne_hx(x)
    
    # prediction de l'état
    x_pred = f(x_est, u)
    P_pred = A @ P_est @ A.T + B @ Q_est @ B.T
    
    # prediction de l'observation
    Y_predit = h(x_pred)
    
    # correction de la prediction
    K     = P_pred @ H.T @ inv(H @ P_pred @ H.T + P_Y_est)
    x_est = x_pred + K @ (Y_mesure - Y_predit)
    P_est = P_pred - K @ H @ P_pred
    P_est = 0.5 * (P_est + P_est.T) # ensure symetry
    
    return x_est, P_est

# Exemple 

Robot sur un plan 2D : $ X_t = [ x_t, y_t, \theta_t ]^T $ et $ u_t = [x_u, y_u, \theta_u] $

$ f(X_t,u_t) = \left( \begin{array}{c} x_t + x_u \cos(\theta_t) - y_u \sin(\theta_t) \\ y_t + x_u \sin(\theta_t) + y_u \cos(\theta_t) \\ \theta_t + \theta_u \\ \end{array} \right) $ &emsp; et &emsp; $ h(X_t) = \left( \begin{array}{c} \sqrt{(x_k-x_t)^2+(y_k-y_t)^2} \\ atan2(\frac{y_k-y_t}{x_k-x_t}) - \theta_t \\ \end{array} \right) $

## Environnement

In [1]:
# composes 2 transformations
def T_compose(T_ab, T_bc):
    somme = AngleWrap(T_ab[2] + T_bc[2])
    s     = sin(T_ab[2])
    c     = cos(T_ab[2])
    T_ac  = T_ab[0:2] + np.array([[c, -s],[s, c]]) @ T_bc[0:2]
    T_ac  = np.append(T_ac, somme)
    return T_ac

# calcul l'inverse d'une transformation
def t_inverse(T_ab):
    s    = sin(T_ab[2])
    c    = cos(T_ab[2])
    T_ba = np.array([-T_ab[0]*c - T_ab[1]*s, T_ab[0]*s - T_ab[1]*c, -T_ab[2]])
    return T_ba

def T_inverse(T_ab):
    T_ba = 0.0 * T_ab;
    for t in range(0, T_ba.shape[0], 3):
        T_ba[t:t+3] = t_inverse(T_ab[t:t+3])
    return T_ba

# construit une mesure d'odométrie
def GetRobotControl(k):
    u = np.array([0, 0.025, 0.1 * np.pi/180 * math.sin(3*np.pi*k/nSteps)])
    return u

def GetOdometry(k):
    u           = GetRobotControl(k)
    x_odom_now  = T_compose(x_odom_prev, u)
    u_noise     = np.sqrt(Q_true) @ randn(3)
    x_odom_now  = T_compose(x_odom_now, u_noise)
    x_odom_prev = x_odom_now
    return xnow



def SimulateWorld(k):
    u         = GetRobotControl(k)
    x_true    = T_compose(x_true, u)
    x_true[2] = AngleWrap(x_true[2])

## Initialisation

In [ ]:
nSteps = 10000

# génère des amers
amers = 140 * rand(2,20) - 70
plt.cla()
plt.plot(amers[0,:], amers[1,:],'*g', markersize=15)
plt.text(amers[0,:] + 2, amers[1,:] + 2,'amer', color='green', fontsize=15)

# vraie covariance des erreurs utilisées pour simuler les mouvements du robot
Q_true  = np.diag([0.01, 0.01, 1*math.pi/180]) ** 2
PY_true = diag([2.0, 3*math.pi/180]) ** 2

# erreurs estimée pour le filtre de Kalman
Q_est  = Q_true  + Q_true * (8/10)
PY_est = PY_true - PY_true * (8/10)

# conditions initiales
x_true      = np.array([1,-40,-math.pi/2])
x_odom_prev = x_true
x_est = np.array([0,0,0])
P_est = np.diag([1,1,1*(math.pi/180)**2])

## Main Loop

In [ ]:
for k in range(1, nSteps):
    
    # simulation
    SimulateWorld(k)
    
    # trouve le contrôle en soustrayant l'actuelle et la précédente valeur d'odométrie
    x_odom_now  = GetOdometry(k)
    u           = T_compose(T_inverse(x_odom_prev), x_odom_now)
    x_odom_prev = x_odom_now
    
    # observe a random feature
    [y_mesure, iFeature] = GetObservation(k)
    
    def h(x, amer):
        distance = norm(amer[0:2] - x[0:2])
        z        = np.array([distance, AngleWrap(atan2(distance[1], distance[0]) - x[2])])
        return z

    def f(x_est, u):
        return x_pred = T_compose(x_est, u)
    
    # mise à jour avec Kalman
    x_est, P_est = Kalman_NL_update(x_est, P_est, Y_mesure, f, h)
            
    # plot every 200 updates
    if (k-2) % 75 == 0:
        DoVehicleGraphics(x_est, P_est[0:2,0:2],8,[0,1])